In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

url = 'https://www.booking.com/searchresults.html?ss=India&ssne=India&ssne_untouched=India&efdco=1&label=gen173rf-1FCAEiBWhvdGVsKIICOOgHSDNYA2hsiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBogIQc2VhcmNoLmJyYXZlLmNvbagCA7gCgrKdsgbAAgHSAiRhMjBjNzY2NS0xNDJiLTQ0OTAtOTExOS1iZjM1YTZjMjQxNjjYAgXgAgE&sid=ba5493355b758883a61f8556fe47c06a&aid=304142&lang=en-us&sb=1&src_elem=sb&src=searchresults&dest_id=98&dest_type=country&checkin=2024-05-22&checkout=2024-05-23&group_adults=1&no_rooms=1&group_children=0'

HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'}

# Use Selenium to simulate clicking the "Load more results" button
driver = webdriver.Chrome()  # Make sure ChromeDriver is in your PATH
driver.get(url)

# Implement an explicit wait to handle dynamic content
wait = WebDriverWait(driver, 10)

# Click the "Load more results" button until it is no longer available
load_more_button_selector = '#bodyconstraint-inner > div:nth-child(8) > div > div.af5895d4b2 > div.df7e6ba27d > div.bcbf33c5c3 > div.dcf496a7b9.bb2746aad9 > div.d4924c9e74 > div.c82435a4b8.f581fde0b8 > button'

while True:
    try:
        load_more_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, load_more_button_selector)))
        load_more_button.click()
        time.sleep(2)  # Wait for new hotels to load
    except:
        break

# Get the HTML content after all hotels are loaded
webpage = driver.page_source
driver.quit()

soup = BeautifulSoup(webpage, 'html.parser')
section = soup.find('div', class_='d4924c9e74')
hotels = section.find_all('div', attrs={'data-testid': 'property-card'})
hotel_data = []  # List to store hotel data as dictionaries

for hotel in hotels:
    topic = {}  # Initialize an empty dictionary for each hotel
    topic['name'] = hotel.img['alt'] if hotel.img else None
    topic['Location'] = hotel.find('span', {'data-testid': 'address'}).text if hotel.find('span', {'data-testid': 'address'}) else None
    topic['Room_Type'] = hotel.find('h4', class_='abf093bdfe e8f7c070a7').text.strip() if hotel.find('h4', class_='abf093bdfe e8f7c070a7') else None
    topic['Bed'] = hotel.find('div', class_='fc367255e6').text.strip() if hotel.find('div', class_='fc367255e6') else None
    topic['Price'] = hotel.find('span', attrs={'data-testid': 'price-and-discounted-price'}).text if hotel.find('span', attrs={'data-testid': 'price-and-discounted-price'}) else None
    topic['tax'] = hotel.find('div', attrs={'data-testid': 'taxes-and-charges'}).text if hotel.find('div', attrs={'data-testid': 'taxes-and-charges'}) else None
    topic['Reviews'] = hotel.find('div', class_='a3b8729ab1 d86cee9b25').find('div', class_='ac4a7896c7').get_text(strip=True) if hotel.find('div', class_='a3b8729ab1 d86cee9b25') else None
    topic['link'] = 'https://www.booking.com' + hotel.a['href'] if hotel.a else None
    hotel_data.append(topic)


In [27]:
df_csv = pd.DataFrame(hotel_data)
df_csv.to_csv('hotels1.csv', index=False)

# Save as Excel
df_excel = pd.DataFrame(hotel_data)
df_excel.to_excel('hotels1.xlsx', index=False)